# Exploratory Data Analysis

This notebook explores the data for implementing the approach described in Yang and Malik (2024): "Reinforcement Learning Pair Trading: A Dynamic Scaling Approach". We're using BTCUSDT and ETHUSDT as our trading pair.

## 1. Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import coint, adfuller
import os
%matplotlib inline

# Set style for better visualizations
plt.style.use('seaborn-v0_8-whitegrid')

# Create docs directory for saving plots
os.makedirs('docs', exist_ok=True)

In [ ]:
print("Files in current directory:", os.listdir())
print("Current working directory:", os.getcwd())

## 2. Load Data

We'll use the BTCUSDT and ETHUSDT 1-minute data collected from Binance.US API.

In [ ]:
# Load the data
try:
    # Go up one directory level and then into 'data'
    btcusdt = pd.read_csv('../data/btcusdt_1m_raw.csv')
    ethusdt = pd.read_csv('../data/ethusdt_1m_raw.csv')

    # Basic data exploration
    print("BTCUSDT data shape:", btcusdt.shape)
    print("ETHUSDT data shape:", ethusdt.shape)

    print("\nBTCUSDT first 5 rows:")
    display(btcusdt.head())
    
    print("\nETHUSDT first 5 rows:")
    display(ethusdt.head())
except FileNotFoundError as e:
    print(f"Data files not found: {e}")
    print("Please run the data collection script first.")

## 3. Data Preparation

Let's prepare the data for analysis by ensuring datetime indexes and proper data types.

In [ ]:
# Convert timestamp to datetime if not already done
btcusdt['datetime'] = pd.to_datetime(btcusdt['datetime'])
ethusdt['datetime'] = pd.to_datetime(ethusdt['datetime'])

# Set datetime as index
btcusdt.set_index('datetime', inplace=True)
ethusdt.set_index('datetime', inplace=True)

# Check data info
print("BTCUSDT date range:", btcusdt.index.min(), "to", btcusdt.index.max())
print("ETHUSDT date range:", ethusdt.index.min(), "to", ethusdt.index.max())

## 4. Basic Statistics

In [ ]:
# Calculate basic statistics
btc_stats = btcusdt['close'].describe()
eth_stats = ethusdt['close'].describe()

stats_df = pd.DataFrame({
    'BTC-USDT': btc_stats,
    'ETH-USDT': eth_stats
})

# Display statistics
display(stats_df)

# Calculate returns
btcusdt['returns'] = btcusdt['close'].pct_change()
ethusdt['returns'] = ethusdt['close'].pct_change()

# Calculate return statistics
btc_return_stats = btcusdt['returns'].describe()
eth_return_stats = ethusdt['returns'].describe()

return_stats_df = pd.DataFrame({
    'BTC-USDT Returns': btc_return_stats,
    'ETH-USDT Returns': eth_return_stats
})

# Display return statistics
display(return_stats_df)

## 5. Price Visualization

In [ ]:
# Visualize price movements
plt.figure(figsize=(14, 7))

# Create two y-axes
ax1 = plt.gca()
ax2 = ax1.twinx()

# Plot BTC on left axis
btcusdt['close'].plot(ax=ax1, color='blue', label='BTC-USDT')
ax1.set_ylabel('BTC Price (USDT)', color='blue')

# Plot ETH on right axis
ethusdt['close'].plot(ax=ax2, color='orange', label='ETH-USDT')
ax2.set_ylabel('ETH Price (USDT)', color='orange')

# Add title and legend
plt.title('BTC-USDT vs ETH-USDT Price Movement', fontsize=14)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Save and show
plt.savefig('docs/price_movement.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot returns
plt.figure(figsize=(14, 7))
plt.plot(btcusdt['returns'].dropna(), label='BTC-USDT Returns', alpha=0.7)
plt.plot(ethusdt['returns'].dropna(), label='ETH-USDT Returns', alpha=0.7)
plt.title('BTC-USDT vs ETH-USDT Returns', fontsize=14)
plt.ylabel('Returns')
plt.legend()
plt.savefig('docs/returns.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Correlation and Cointegration Analysis

In [ ]:
# Ensure we're working with common timestamps
common_index = btcusdt.index.intersection(ethusdt.index)
btc_aligned = btcusdt.loc[common_index]['close']
eth_aligned = ethusdt.loc[common_index]['close']

# Calculate price correlation
price_correlation = btc_aligned.corr(eth_aligned)
print(f"Price correlation between BTC-USDT and ETH-USDT: {price_correlation:.4f}")

# Calculate returns correlation
btc_returns_aligned = btcusdt.loc[common_index]['returns'].dropna()
eth_returns_aligned = ethusdt.loc[common_index]['returns'].dropna()
common_return_index = btc_returns_aligned.index.intersection(eth_returns_aligned.index)
returns_correlation = btc_returns_aligned.loc[common_return_index].corr(eth_returns_aligned.loc[common_return_index])
print(f"Returns correlation between BTC-USDT and ETH-USDT: {returns_correlation:.4f}")

# Test for cointegration
score, pvalue, _ = coint(btc_aligned.dropna(), eth_aligned.dropna())
print(f"Cointegration p-value: {pvalue:.4f}")
print(f"Pairs are {'cointegrated' if pvalue < 0.05 else 'not cointegrated'} at 5% significance level")

# Plot correlation
plt.figure(figsize=(10, 8))
plt.scatter(btc_aligned, eth_aligned, alpha=0.3)
plt.title(f'BTC-USDT vs ETH-USDT (Correlation: {price_correlation:.4f})', fontsize=14)
plt.xlabel('BTC-USDT Price')
plt.ylabel('ETH-USDT Price')
plt.grid(True, alpha=0.3)
plt.savefig('docs/correlation.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Price Ratio and Spread Analysis

For pair trading, we need to examine the spread or ratio between the two assets.

In [ ]:
# Calculate price ratio (BTC/ETH)
price_ratio = btc_aligned / eth_aligned

# Visualize price ratio
plt.figure(figsize=(14, 6))
plt.plot(price_ratio)
plt.title('BTC-USDT to ETH-USDT Price Ratio', fontsize=14)
plt.ylabel('BTC/ETH Ratio')
plt.axhline(y=price_ratio.mean(), color='r', linestyle='-', label=f'Mean: {price_ratio.mean():.2f}')
plt.axhline(y=price_ratio.mean() + 2*price_ratio.std(), color='g', linestyle='--', label='Upper Threshold (2σ)')
plt.axhline(y=price_ratio.mean() - 2*price_ratio.std(), color='g', linestyle='--', label='Lower Threshold (2σ)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/price_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate log price ratio (log(BTC) - log(ETH))
log_ratio = np.log(btc_aligned) - np.log(eth_aligned)

# Test log ratio for stationarity (Augmented Dickey-Fuller test)
adf_result = adfuller(log_ratio.dropna())
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")
print(f"The log ratio is {'stationary' if adf_result[1] < 0.05 else 'non-stationary'} at 5% significance level")

# Visualize log ratio
plt.figure(figsize=(14, 6))
plt.plot(log_ratio)
plt.title('Log Price Ratio (log(BTC) - log(ETH))', fontsize=14)
plt.ylabel('Log Ratio')
plt.axhline(y=log_ratio.mean(), color='r', linestyle='-', label=f'Mean: {log_ratio.mean():.4f}')
plt.axhline(y=log_ratio.mean() + 2*log_ratio.std(), color='g', linestyle='--', label='Upper Threshold (2σ)')
plt.axhline(y=log_ratio.mean() - 2*log_ratio.std(), color='g', linestyle='--', label='Lower Threshold (2σ)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/log_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Z-Score Calculation for Pair Trading

Following the methodology in the paper, we'll calculate the z-score using a rolling window approach.

In [ ]:
# Calculate z-score with rolling window (as described in the paper)
window_size = 900  # As mentioned in paper
rolling_mean = log_ratio.rolling(window=window_size).mean()
rolling_std = log_ratio.rolling(window=window_size).std()
z_score = (log_ratio - rolling_mean) / rolling_std

# Visualize z-score
plt.figure(figsize=(14, 6))
plt.plot(z_score.dropna())
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=1.8, color='r', linestyle='--', label='Open Threshold (1.8)')
plt.axhline(y=-1.8, color='r', linestyle='--')
plt.axhline(y=0.4, color='g', linestyle='--', label='Close Threshold (0.4)')
plt.axhline(y=-0.4, color='g', linestyle='--')
plt.title('Z-score of BTC-USDT and ETH-USDT Log Ratio', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/zscore.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Trading Signal Analysis

Identify potential trading signals based on the z-score thresholds.

In [ ]:
# Analyze potential trading opportunities
trading_zones = pd.DataFrame(index=z_score.dropna().index)
trading_zones['z_score'] = z_score.dropna()
trading_zones['long_entry'] = z_score.dropna() < -1.8  # Long when z-score is below lower threshold
trading_zones['short_entry'] = z_score.dropna() > 1.8  # Short when z-score is above upper threshold
trading_zones['long_exit'] = (z_score.dropna() > -0.4) & (z_score.dropna() < 0.4)  # Exit long when z-score is back in normal range
trading_zones['short_exit'] = (z_score.dropna() > -0.4) & (z_score.dropna() < 0.4)  # Exit short when z-score is back in normal range

# Count potential signals
entry_signals = trading_zones['long_entry'].sum() + trading_zones['short_entry'].sum()
exit_signals = trading_zones['long_exit'].sum() + trading_zones['short_exit'].sum()

print(f"Number of potential entry signals: {entry_signals}")
print(f"Number of potential exit signals: {exit_signals}")
print(f"Long entry signals: {trading_zones['long_entry'].sum()}")
print(f"Short entry signals: {trading_zones['short_entry'].sum()}")

# Distribution of signals over time
plt.figure(figsize=(14, 7))
plt.plot(trading_zones['z_score'], color='gray', alpha=0.6, label='Z-score')
plt.scatter(trading_zones[trading_zones['long_entry']].index, 
            trading_zones[trading_zones['long_entry']]['z_score'],
            color='green', marker='^', label='Long Entry')
plt.scatter(trading_zones[trading_zones['short_entry']].index, 
            trading_zones[trading_zones['short_entry']]['z_score'],
            color='red', marker='v', label='Short Entry')
plt.axhline(y=0, color='black', linestyle='-')
plt.axhline(y=1.8, color='r', linestyle='--')
plt.axhline(y=-1.8, color='r', linestyle='--')
plt.axhline(y=0.4, color='g', linestyle='--')
plt.axhline(y=-0.4, color='g', linestyle='--')
plt.title('Trading Signals Based on Z-score', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('docs/trading_signals.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Trading Opportunity Distribution

Let's analyze the distribution of trading opportunities over time.

In [ ]:
# Group by date to analyze trading opportunities by day
trading_zones['date'] = trading_zones.index.date
daily_opportunities = trading_zones.groupby('date')[['long_entry', 'short_entry']].sum()

# Visualize daily opportunities
plt.figure(figsize=(14, 6))
daily_opportunities.plot(kind='bar', stacked=True)
plt.title('Daily Trading Opportunities', fontsize=14)
plt.ylabel('Number of Signals')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('docs/daily_opportunities.png', dpi=300, bbox_inches='tight')
plt.show()

# Calculate opportunity statistics by hour of day
trading_zones['hour'] = trading_zones.index.hour
hourly_opportunities = trading_zones.groupby('hour')[['long_entry', 'short_entry']].sum()

# Visualize hourly opportunities
plt.figure(figsize=(14, 6))
hourly_opportunities.plot(kind='bar', stacked=True)
plt.title('Trading Opportunities by Hour of Day', fontsize=14)
plt.ylabel('Number of Signals')
plt.xlabel('Hour of Day')
plt.xticks(range(24))
plt.grid(True, alpha=0.3)
plt.savefig('docs/hourly_opportunities.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Summary and Next Steps

Based on our exploratory data analysis, let's summarize what we've learned about our BTCUSDT and ETHUSDT pair for trading:

In [ ]:
# Summary statistics
summary = {
    'Price Correlation': price_correlation,
    'Returns Correlation': returns_correlation,
    'Cointegration p-value': pvalue,
    'Is Cointegrated (5% level)': pvalue < 0.05,
    'Log Ratio Stationarity p-value': adf_result[1],
    'Is Log Ratio Stationary (5% level)': adf_result[1] < 0.05,
    'Number of Long Entry Signals': trading_zones['long_entry'].sum(),
    'Number of Short Entry Signals': trading_zones['short_entry'].sum(),
    'Trading Days Covered': len(daily_opportunities),
    'Mean BTC/ETH Ratio': price_ratio.mean(),
    'BTC/ETH Ratio Std Dev': price_ratio.std()
}

# Convert to DataFrame for nice display
summary_df = pd.DataFrame(list(summary.items()), columns=['Metric', 'Value'])
display(summary_df)

### Next Steps:

1. **Traditional Pair Trading Implementation**: Implement a traditional pair trading strategy using fixed thresholds as a baseline.

2. **Reinforcement Learning Environment**: Design an RL environment for pair trading as described in the paper, including:
   - Observation space: Position, Spread, and Zone
   - Action space: Continuous value between -1 and 1 for investment quantity
   - Reward structure: Portfolio reward, action reward, and transaction punishment

3. **RL Model Training**: Train different RL models (A2C, PPO, DQN) and compare their performance.

4. **Performance Evaluation**: Compare the performance of traditional pair trading vs. RL-based approaches.

5. **Hyperparameter Tuning**: Optimize the models' performance through hyperparameter tuning.